# export tiles for contextal layers


In [1]:
import ee
import geemap

In [2]:
# Trigger the authentication flow.
# ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate(auth_mode='notebook')


Successfully saved authorization token.


In [2]:
# Initialize the library.
ee.Initialize()

In [3]:
def exportMapTask(asset: ee.Image, asset_name: str, gcbucket: str,
                #region: ee.Geometry, min_zoom: int, max_zoom: int,
                min_zoom: int, max_zoom: int, 
                env: str = 'staging', key: str = 'year'):
    """
    Export the data to GEE.

    Parameters
    ----------
    asset : ee.Image
        The image to export.
    asset_name : str
        The name of the asset.
    gcbucket : str
        The name of the GCS bucket.
    geometry_collection : ee.GeometryCollection
        The geometry collection to export the data.
    
    Returns
    -------
    A task
    """
    
    
    return ee.batch.Export.map.toCloudStorage(
            image = asset,
            description = f'{asset_name}',
            path = f'{env}/tilesets/{asset_name}',
            bucket = gcbucket,
            minZoom = min_zoom,
            maxZoom = max_zoom,
            writePublicTiles = False,
            skipEmptyTiles = True,
            fileFormat = 'png'#,
            #region = region
            )

### Data Layer: Fishing intensity associated with mangroves
#### Visualize the data prior exporting it

In [4]:
gcs_bucket = 'mangrove_atlas'

dataset = f'fishing_intensity_mangroves'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
image = ee.Image("projects/mangrove-atlas-246414/assets/fishing_intensity_mangroves")

# exagerated image overviews for zoomed out views
#overviews = image_extent.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
visoptions =  {"palette": ['B7E6A5', 'ca5268'], "min": 1, "max": 2}

styled_image = image.sldStyle("""
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
            <ColorMapEntry color="#F3E79B" quantity="50" label="0 - 50"/> +
            <ColorMapEntry color="#FAC484" quantity="200" label=">50 - 200"/> +
            <ColorMapEntry color="#F0746E" quantity="700" label=">200 - 700"/> +
            <ColorMapEntry color="#B9257A" quantity="2000" label=">700 - 2000"/> +
            <ColorMapEntry color="#701A64" quantity="3000000" label=">2000"/> +
        </ColorMap>
    </RasterSymbolizer>
""")
# exagerated image overviews for zoomed out views
overviews = styled_image.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 2).selfMask()

Map = geemap.Map(center=( 3.3, 117.4), zoom=9, basemap='CartoDB.PositronNoLabels')

Map.addLayer(styled_image, {}, f"{dataset}", True, 1)
Map.addLayer(overviews, {}, f"{dataset}", True, 1)

Map

Map(center=[3.3, 117.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

#### Export to tiles from GEE to GCS

In [6]:
task = exportMapTask(overviews, f'{dataset}_updated', gcs_bucket, 
                     1, 3)
# Create a list of tasks to export the map tiles
task2 = exportMapTask(styled_image, f'{dataset}_updated', gcs_bucket, 
                     4, 12)
# execute tasks in batches due to gee limits
task.start()
task2.start()
